# Probability-Based Explanation of the Core SLAM Algorithm

### Formulation and Structure of the SLAM Problem

**1. Preliminaries**

This document assumes the reader has a basic working knowledge of probability theory, specifically: random variables, distributions, joint distributions, conditional probability, and Bayes' Theorem. If you don't have these, learn them quickly and then come back. It'll be more fun that way.

We define the following sets of information for the SLAM problem

- $X_{0:k} = \{ x_0, x_1, …, x_k \}$ - The set of all robot locations
- $U_{0:k} = \{ u_0, u_1, …, u_k \}$ - The set of all control inputs
- $m = \{ m_0, m_1, …, m_k\}$ - The set of all landmark locations. When only one landmark is present or the specific landmark doesn't matter, we'll just use m
- $Z_{0:k} = \{ z_1, z_2, …, z_k \}$ - The set of all landmark observations

**2. Probabilistic SLAM**

The SLAM problem requires that we get a joint probability distribution encoding for the information that we care about. We then force that distribution to converge in order to extract that information. Since the robot follows a Markov process, each step is solely dependant on the previous step. Therefore, we can compute the probability distribution for all $k$ recursively via the following expression:

$$
P(x_k, m|Z_{0:k}, U_{0:k}, x_0)
$$

Here, $x_k$ represents the probability distribution of the robot's location at time $k$ and $m$ represents the landmark probability distribution, also at time $k$, conditioned on all observations made so far. The control vector sequence, $U_{0:k}$, and start point, $x_0$, are jointed into this distribution as they both will impact the locations of $x_k$ and therefore $m$. In addition, it's logical to assume that where the robot ends up at any given time in relation to the landmark estimates will also affect the control vector sequence. TLDR: All the probability distributions are connected in some way.

Tying back into our what was discussed earlier, as $k$ approaches infinity, we expect the distributions for $xk, m$ to converge i.e. have a really big spike in one place and very little probability of being anywhere else. This means that we need to get this joint distribution to converge as the number of observations increases.

To do this we get the prior distribution:

$$
P(x_{k-1}, m|Z_{0:k-1}, U_{0:k-1})
$$

Apply a control signal $u_k$, take an observation $z_k$, and use that new information to compute the new distribution.
To do this, we need to get some context first:


The **motion model** describes the probability of the robot location given prior location and control inputs:

>(1)
>$$
>P(x_k|x_{k-1}, u_k)
>$$

The **observation model** describes the probability of making an observation given the map and vehicle position:

>(2)
>$$
>P(z_k|x_k, m)
>$$


The SLAM algorithm uses a two step process.

First, it makes the **time update**, which updates the prediction of the position and map given all prior landmark observations $Z_{0:{k-1}}$ and all motion commands including the most recent command, $u_k$:

> (3)
>$$
>P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0) = \int P(x_k| x_{k-1}, u_k) \times P(x_{k-1}, m|Z_{0:k-1}, U_{0:k-1}, x_0) \: dx_{k-1}
>$$


Looking at the left side, we are getting the joint distribution $P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0)$. From what we asserted above, we know that we can get the probability distribution $P(x_j, m|Z_{0:j}, U_{0:j}, x_0)$ for any j greater than 0, including k - 1. This is where the recursion comes in. 

From here, we can use the motion model to merge in the $P(x_k|x_{k-1}, u_k)$ distribution, this gives us the marginal joint probability distribution $P(x_k|x_{k-1}, x_{k-1}, m|Z_{0:k-1})$ jointed in with some other stuff we don't care about ($U_{0:k}, x_0$). Now the integral becomes obvious, if we integrate with respect to $x_{k-1}$, it gives us the overall probability distribution of $x_k$, no longer conditioned on $x_{k-1}$, and also jointed in with the same stuff we don't currently care about:

$$
P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0)
$$

This makes intuitive sense as $x_{k-1}$ has its own distribution; the probability distribution for $x_k$ given a specific $x_{k-1}$ does not necessarily reflect the entire distribution of $x_k$.


The next step is to make an observation and further tune our distributions. We use the following **measurement update** formula to do this:

> (4)
>$$
>P(x_k, m|Z_{0:k}, U_{0:k}, x_0) = \frac{P(z_k|x_k,m)P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0)}{P(z_k|Z_{0:k-1},U_{0:k})}
>$$

Specifically, we use the new time-update distribution $P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0)$, and we merge this (as per Bayes' theorem) with the observation model $P(z_k|x_k, m)$.

This input from the observation model encodes new information about the correlations of robot and map positions which gets factored into our posterior (LHS of (4)).

We can view the map building problem as computing the distribution $P(m|X_{0:k}, Z_{0:k}, U_{0:k})$, and we can view the localization problem as computing the distribution $P(x_k|Z_{0:k}, U_{0:k}, m)$. Solving these sub-problems separately is only possible when either the position or map is known respectively.

The dependence of these predictions on each other is reflected in the convergence properties of the joint posterior distribution.

###  Convergence of the Joint Distribution


Earlier, we discussed the following statement:
>**1. Correlations between landmark estimates increase monotonically with the number of observations taken.**

We also mentioned that given ododmetry data is the primary source of error, the relative distances between points is fairly accurate. We can now put these two statements together to explain how SLAM algorithms work in a general sense.

In probabalistic terms, what happens is that the joint probability density $P(m)$ becomes monotonically more peaked as more observations are made. In other words, each landmark ends up having a very high chance of being in a given area and a very low chance of being in any other. 

To explain why this happens, consider this. We are taking more and more successive (nearly, as observation errors will technically be correlated through vehicle motions) independant measurements of each landmark. Since the relative distances between landmarks don't depend on the vehicle position, and since over a large number of observations, the mean noise across observations will tend to approach 0 (as per what was discussed in the preceeding section), what we see is that the relative distances between landmarks becomes more and more clearly defined.

Given that we know the distances between landmarks well, and we are able to obtain landmark probability distributions as a function of observation and vehicle location distributions. If we assert that landmark $m_i$ exists in a area of low probability, and we have a good idea of the distances between landmarks, then it follows that every other landmark also exists in an area of low probability, this is by definition very unlikely. Likewise, if we assert that $m_i$ exists in a area of high probability, then it follows that every other landmark also exists in an area of high probability, this is by definition very likely. These assertions are driven by the robot estimating it's position and then taking an observation. If the assertion yields a very low likelihood of being congruent with the map, either the map is wrong (more likely at the beginning) or the robot is wrong (more likely later on).

We can combine these landmark distributions together to get the joint distribution for the map, $P(m)$. By conditioning on any one landmark, we find that over time, the joint distribution becomes more and more peaked at the area the landmark is most likely to exist and less peaked everywhere else. 

What we are effectively doing is using observations to update this joint map distribution $P(m)$, jointed in with the vehicle state distribution $x_k$. By recursively applying Bayesian updates, the uncertainty in these estimates decreases over time, driving the convergence of the joint posterior distribution.

$$
P(x_k, m|Z_{0:k}, U_{0:k}, x_0)
$$

Since our system satisfies the Markov property, we can drop the conditioning on historical variables as they are already encoded in our robot's state and the map distributions:

$$
P(x_k, m|z_k)
$$


### What This Means

SLAM isn't really one single algorithm. It's actually a family of algorithms. Here, "Core SLAM Algorithm" is really only the various abuses of probability theory that are shared among all solutions to the SLAM problem.

The difference between implementations comes down to how the motion (1) and observation (2) models are obtained and how the time-update (3) and measurement-update (4) distributions are computed. Appropriate solutions to the SLAM problem must do these things in a way that is coherent with the logic outlined above.

Real-world use-cases necessitate that data is collected through different means depending on the situation. Real-world environments tend to be non-static. As such, real-world solutions to the SLAM problem tend to have a significant amount of added complexity to account for these external factors. We will take a closer look at some of these in the following sections.

For anyone looking to really die on this hill or otherwise vibe-check what I'm saying, I'd recommend reading [this bad boy here](../resources/Durrant-Whyte_Bailey_SLAM-tutorial-I.pdf).

If I made an error somewhere, please let me know.

Thinking about probability distributions can be tiresome. If you are feeling a bit tired, please enjoy this [complementary intermission](https://www.youtube.com/watch?v=lMoloUg8QpY).
